# Main Library

In [1]:
# Reading Data
import json
import numpy as np
import string, random

# Data Preprocessing
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Building Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from keras.models import load_model

# Dowmload Model
import pickle

# Loading Models

In [2]:
words = pickle.load(open(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V1\\words.pkl", "rb"))
classes = pickle.load(open(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V1\\classes.pkl", "rb"))
corpus = pickle.load(open(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V1\\corpus.pkl", "rb"))

model = load_model(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V1\\model_chat_bot.h5")

# Reading Data

In [ ]:
file = open(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V1\data.json").read()
data = json.loads(file)

ignore = ["?", "!"]

word_Response = []
classes = []
corpus = []

lemma = WordNetLemmatizer()

for record in data["info"]:
    for req in record["requests"]:
        word =  word_tokenize(req)
        word = [lemma.lemmatize(w.lower()) for w in word if (w not in ignore)]
        word_Response.extend(word)
        t = record["title"]
        corpus.append((word, t))
        if t not in classes: classes.append(t)
            

words = list(set(word_Response))
words = sorted(words)
classes = sorted(classes)

pickle.dump(words, open(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V1\\words.pkl", "wb"))
pickle.dump(classes, open(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V1\\classes.pkl", "wb"))
pickle.dump(corpus, open(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V1\\corpus.pkl", "wb"))

# Digital Representation

In [ ]:
data_set = []

for request, class_r in corpus:
    input_vec = [0] * len(words)
    output_vec = [0] * len(classes)
    
    for w in request:
        input_vec[words.index(w)] = 1
        
    output_vec[classes.index(class_r)] = 1
    
    data_set.append([input_vec, output_vec])

random.shuffle(data_set)
data_set = np.array(data_set, dtype=object)

X = list(data_set[:, 0])
Y = list(data_set[:, 1])

Y[:3]

# Building Model

In [ ]:
def create_model(input_size, output_size):
    model = Sequential([
        Dense(128, input_shape=(input_size,), activation="relu"),
        Dropout(0.5),
        Dense(64, activation="relu"),
        Dropout(0.5),
        Dense(output_size, activation="softmax")])
    
    return model

model = create_model(len(words), len(classes))
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01), loss="categorical_crossentropy", metrics=["accuracy"])

# model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.8, decay=1e-5, nesterov=True), loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(X, Y, epochs=200, batch_size=3, verbose=1)

model.save(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V1\\model_chat_bot.h5")

# -----------------------------------------------------------------------------------------------

# Make Prediction

In [28]:
sen = "Hello, How are you?"

def sen_words(sen):
    swords = word_tokenize(sen)
    swords = [w.lower() for w in swords]
    return swords

def digitize(sen, words):
    input_vec = [0] * len(words)
    swords = sen_words(sen)
    
    for sw in swords:
        if sw in words: input_vec[words.index(sw)] = 1
    return input_vec

def predict_class(sen, words, classes, model):
    sample = digitize(sen, words)
    value = model.predict(np.array([sample]))[0]
    
    return classes[np.argmax(value)]

def get_response(sen, words, classes, model):
    file = open(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V1\data.json").read()
    data = json.loads(file)
    tag = predict_class(sen, words, classes, model)
    for record in data["info"]:
        if record["title"] == tag: return random.choice(record["responses"])
        
get_response(sen, words, classes, model)

1/1 [==============================] - 0s 14ms/step


'Good to see you again'